In [ ]:
#import my model from my drive
from google.colab import drive
drive.mount('/content/gdrive')
!git clone https://github.com/n00bmaster68/ApplyingSODinAutoLabelling

#check whether device has GPU
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
!nvidia-smi -L

In [ ]:
import os
import time
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import numpy as np
import cv2
from PIL import Image
import glob
 
from ApplyingSODinAutoLabelling.data_loader import RescaleT
from ApplyingSODinAutoLabelling.data_loader import CenterCrop
from ApplyingSODinAutoLabelling.data_loader import ToTensor
from ApplyingSODinAutoLabelling.data_loader import ToTensorLab
from ApplyingSODinAutoLabelling.data_loader import SalObjDataset

from ApplyingSODinAutoLabelling.model import BASNet

In [ ]:
def normPRED(d):
	ma = torch.max(d)
	mi = torch.min(d)

	dn = (d-mi)/(ma-mi)
	return dn

In [ ]:
def save_output(image_name,pred,d_dir):
	predict = pred
	predict = predict.squeeze()
	predict_np = predict.cpu().data.numpy()

	im = Image.fromarray(predict_np*255).convert('RGB')
	img_name = image_name.split("/")[-1]
	image = io.imread(image_name)
	imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)
	# imo.show()
	getCoordinate(imo)

In [ ]:
def getCoordinate(imo):
	opencvImage = cv2.cvtColor(np.array(imo), cv2.COLOR_RGB2BGR)
	height, width, _ = opencvImage.shape
	lower = [np.mean(opencvImage[:,:,i] - np.std(opencvImage[:,:,i])/3 ) for i in range(3)]
	upper = [250, 250, 250]

	# create NumPy arrays from the boundaries
	lower = np.array(lower, dtype="uint8")
	upper = np.array(upper, dtype="uint8")

	# find the colors within the specified boundaries and apply
	mask = cv2.inRange(opencvImage, lower, upper)
	output = cv2.bitwise_and(opencvImage, opencvImage, mask=mask)

	ret,thresh = cv2.threshold(mask, 40, 255, 0)

	contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

	largest_contour = max(contours, key = cv2.contourArea)
	x,y,w,h = cv2.boundingRect(largest_contour)
	print(x/width, y/height, w/width, h/height)
	print(f"x:{x}, y:{y}, hW:{h}, wW:{w}")
	print(f"Height:{height}, Weight:{width}")

In [ ]:
# --------- 1. get image path and name ---------
image_dir = 'ApplyingSODinAutoLabelling/test_data/test_images/'
prediction_dir = 'ApplyingSODinAutoLabelling/test_data/test_results/'
model_dir = '/content/gdrive/MyDrive/basnet.pth'
img_name_list = glob.glob(image_dir + '*.jpg')

In [ ]:
# --------- 2. dataloader ---------
#1. dataload
test_salobj_dataset = SalObjDataset(img_name_list = img_name_list, lbl_name_list = [],transform=transforms.Compose([RescaleT(256),ToTensorLab(flag=0)]))
test_salobj_dataloader = DataLoader(test_salobj_dataset, batch_size=1,shuffle=False,num_workers=1)
	

In [ ]:
	# --------- 3. model define ---------
print("...load BASNet...")
net = BASNet(3,1)
net.load_state_dict(torch.load(model_dir))
if torch.cuda.is_available():
  net.cuda()	
net.eval()

In [ ]:
# --------- 4. inference for each image ---------
for i_test, data_test in enumerate(test_salobj_dataloader):
  start = time.time()	
  print("\n inferencing:",img_name_list[i_test].split("/")[-1])
  
  inputs_test = data_test['image']
  inputs_test = inputs_test.type(torch.FloatTensor)
  if torch.cuda.is_available():
    inputs_test = Variable(inputs_test.cuda())
  else:
    inputs_test = Variable(inputs_test)
    
  d1,d2,d3,d4,d5,d6,d7,d8 = net(inputs_test)
  
  # normalization
  pred = d1[:,0,:,:]
  pred = normPRED(pred)	
  
  save_output(img_name_list[i_test],pred,prediction_dir)
  print("Time: ", time.time() - start, "\n")
  # break
  del d1,d2,d3,d4,d5,d6,d7,d8
